<a href="https://colab.research.google.com/github/KKiriri/CNN-Cifar_image-scartch-detect/blob/master/validate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%tensorflow_version 1.x
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras import backend as K
import numpy as np
from keras.models import Model
from keras.models import load_model
from keras.utils import plot_model
import cv2

# MOUNT GOOGLE DRIVE TO READ YOUR MODEL FROM GOOGLE DRIVE
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [18]:

model = load_model('/content/gdrive/My Drive/Colab Notebooks/model.h5')
plot_model(model, to_file='loaded_model.png')
image = cv2.imread('/content/gdrive/My Drive/Colab Notebooks/mlts_103.png',0)
#cv2.imwrite('scratch.png', image)

scratch = np.zeros_like(image)
# SET YOUR PATCH SIZE
PATCH_SIZE = 32
PATCH_SIZE_X = 32
PATCH_SIZE_Y = 32
# LOOP THOUGH THE IMAGE AND DIVIDE THE IMAGE INTO PATCHES TO USE THE MODEL
for y in range(0, image.shape[0]-PATCH_SIZE_Y, PATCH_SIZE_Y):
  for x in range(0, image.shape[1]-PATCH_SIZE_X, PATCH_SIZE_X):
    # CROP THE PATCH
    patch = image[y:y+PATCH_SIZE_Y, x:x+PATCH_SIZE_X]
    # **** DEPENDS ON THE NORMALIZATION IN THE TRAINING TIME YOU SHOULD DO THE SAME NORMALIZATION TO THE TEST PATCH (FOR EXAMPLE 
    # MAYBE YOU JUST NEED TO DIVIDE IT BY 255.)
    # PREPARE PATCH TO PASS IT THE MODEL
    patch = np.reshape(patch.astype('float32')/255., (1, patch.shape[0], patch.shape[1], 1))
    # GET MODEL PREDICTION
    prediction = model.predict(patch)
    #for i in range(len(prediction)):
     # for j in range(len(prediction[0])):
      #   print(prediction[i][j])
    # THE PREDICTION IS A NUMPY ARRAY WHICH EACH ELEMENT SHOWS THE PROBABILITY OF BEING A SCRATCH CORRESPONDING TO THE PIXELS
    # OF THE INPUT PATCH, SO YOU CAN THRESHOLD IT
    prediction[prediction <= 0.045] = 0
    prediction[prediction > 0.045] = 255
    # CONVERT PREDICTION TO uint8
    prediction = prediction.astype('uint8')
    #for i in range(len(prediction)):
      # for j in range(len(prediction[0])):
        #  print(prediction[i][j])
     #PUT THIS PREDICTION IN THE CORRESPONDING PLACE OF scratch MATRIX
    reshapenp=np.reshape(prediction,(PATCH_SIZE_Y,PATCH_SIZE_X))
    scratch[y:y+PATCH_SIZE_Y, x:x+PATCH_SIZE_X] = reshapenp
    #for i in range(len(prediction)):
    #    for j in range(len(prediction[i])):
    #        scratch[y:y+PATCH_SIZE, x:x+PATCH_SIZE] = prediction[i][j]

#for i in range(len(scratch)):
#   for j in range(len(scratch[0])):
#      print(scratch[i][j])

# FINALLY YOU CAN SAVE SCRATCH AS AN IMAGE
cv2.imwrite('/content/gdrive/My Drive/Colab Notebooks/scratch.png', scratch)


True